In [ ]:
import os
import numpy as np
from matplotlib import pyplot as plt
from mod1 import ebm_annual, effect_D_ebm, insolation
from mod1 import response_initial_perturbation, response_CO2_levels
from mod1 import ebm_solar_sensitivity, make_fig_cool_warm
import netCDF4 as nc
from scipy.stats import linregress

# 1/ standard parameters
ifig=1; # if = 1 then make figures
time_kyr=0 # present day insolation (T=0), 115 kyr ago (T=115), etc
resu_solar = insolation(time_kyr,ifig=0) # compute incident solar flux at TOA
S0=1365.0 # solar constant W/m2
folr=0 # OLR: 0: linear (A+BT_s), 1: stefan_boltzmann (epsilon * black body radiation)
falb=2; # 0: constant, 1: Legendre, 2: Legendre_T, 3: NCEP (not yet implemented)
fkdif=0; # turbulent diffusivity (profile) 0: constant, 1: variable (depends on dT/dy)
epsilon_p=0.61; # emissivity at TOA (used only with folr=1)
epsilon_a=0.80; # absorptivity/emissivity (infrared) in the interior of atmosphere (default value is 0.80)
epsilon_o=1.00; # absorptivity/emissivity (infrared) at the sea surface (default value is 1.0)
esun=0.20; # absorptivity of solar radiation (default value is 0.2)
A=200 # W/m2 used when folr=0 (A+BT_s)
B=2.0 # W/m2/K used when folr=0 (A+BT_s) Below you will find the correct values of A and B to be used
D=1.0; # atmospheric eddy diffusivity (W/m2/K) [Note that this is not the correct value to be used...]
#D=2.0e+04; # diffusive coefficient (m2/s/K) when K is variable
nyears=5.0 # time integration (years)
dt=2.0 # days (time step, default value is 2 days)
radp=0.0 # radiative forcing W/m2 = perturbation in OLR (greenhouse effect)
restart=0 # if set to 1 we have to provide initial conditions for SST(phi) and SAT(phi) with phi the latitude
sati=0.0 # initial SAT (used only with restart=1)
ssti=0.0 # initial SST (used only with restart=1)
lambda_T=30.0 # 0: atmos-only run, # 1: atmosphere + ocean mixed layer
ho=5.0 # ocean mixed layer depth (we use a very shallow ocean layer to save computer time
#        --> seek only eq. solutions)
aoht=0.0e15 # Maximum Poleward Ocean Heat Transport (Watts)

param = {'ifig':ifig,'time_kyr':time_kyr,'folr':folr,'falb':falb,'fkdif':fkdif,'A':A,'B':B,\
         'epsilon_p':epsilon_p,'epsilon_a':epsilon_a,'epsilon_o':epsilon_o,'esun':esun,'D':D,\
         'nyears':nyears,'dt':dt,'S0':S0,'radp':radp,'restart':restart,'ssti':ssti,'sati':sati,\
         'resu_solar':resu_solar,'lambda_T':lambda_T,'ho':ho,'aoht':aoht}

In [ ]:
# (a) linear regression of observed OLR onto surface temperature

dirobs=str('data')
ncep_ts = nc.Dataset(dirobs + str('/skt.sfc.mon.1981-2010.ltm.nc'))
ncep_olr = nc.Dataset(dirobs + str('/ulwrf.ntat.mon.1981-2010.ltm.nc'))
ts=np.mean(ncep_ts['skt'],axis=0) # annual mean
olr=np.mean(ncep_olr['ulwrf'],axis=0) # annual mean

# make a linear regression of OLR on TS to find A and B
# You can use "linregress" from "scipy.stats"

# print('Best fit is A = '+ '%0.0f' %AA +' W/m2 and B = '+ '%0.1f' %BB +' W/m2/degC')

In [ ]:
# Before running the model you need to define A and B
# Update then the parameter dictionary "param"
# The value of D will be found below...

#A=
#B=
#param.update({'A':A,'B':B})

In [ ]:
# (b) effect of diffusivity in the annual mean EBM with albedo feedback
param.update({'radp':0.0,'ifig':0,'aoht':0.0})
Darray = np.arange(0.20, 1.05, 0.05)
effect_D_ebm(Darray,param)

In [ ]:
# For which value of D is the meridional temperature contrast about 50 deg.C ?
# Set the value of D here and update the parameters

#D=
#param.update({'D':D})

In [ ]:
# (c) Control experiment

param.update({'aoht':0.0,'ifig':1})
resu0=ebm_annual(param)

In [ ]:
# (d) Same as (c) but with Ocean Heat Transport Convergence
#     maximum poleward oceanic heat transport of 1 PW (1 PW = 10^{15} Watts)

param.update({'aoht':1.0e15})
resu0_aoht=ebm_annual(param)

# compare SAT between the cases with and without OHTC
plt.figure(figsize=(6,4))
plt.plot(resu0['lat'],resu0['sat'][:,-1],'b',label='without OHTC')
plt.plot(resu0_aoht['lat'],resu0_aoht['sat'][:,-1],'r',label='with OHTC')
plt.legend(loc='best',fontsize=14)
plt.xlabel('Latitude',fontsize=14)
plt.ylabel('deg.C',fontsize=14)
plt.xlim(-90,90)
plt.title('Surface Air Temperature',fontsize=14)
plt.show(block=False)

In [ ]:
# (e) Impact of increasing CO2 levels on equilibrium surface temperature
#   do the same with an ocean depth of 700m
param.update({'ifig':0,'aoht':0.0})
response_CO2_levels(param,[1,2,3])

In [ ]:
# (f) Compute the climate feedback parameters for a doubling of CO2
# Compute then the equilibrium climate sensitivity and compare with the nonlinear model
# The initial conditons (on SAT and SST) of the perturbed experiment (with a radiative
# forcing R not equal to zero) are the end state of the previous unperturbed control experiment (resu0)

radp = 5.35*np.log(2) # Radiative forcing for a doubling of CO2 (W/m2)
param.update({'radp':radp,'restart':1,'ssti':resu0['sst'][:,-1],'sati':resu0['sat'][:,-1]})
resu_co2doubling=ebm_annual(param)
resu_co2doubling.keys()

#  compute the feedback parameters from the global averages: satmean, ASRm and OLRm
#  The albedo and planck feedback parameters are given by the linear regression of
#  satmean onto ASRm or OLRm respectively

In [ ]:
# (g) Response to initial perturbation (around equilibrium state obtained with radp=0.0)

delta_T=-20.0 # uniform cooling
param.update({'radp':0.0,'aoht':0.0,'falb':2,'restart':0})
d_olr,d_asr=response_initial_perturbation(param,delta_T)

# compute the feedback parameters using the changes in OLR (d_olr) and ASR (d_asr)
# associated with the initial cooling.

In [ ]:
# Let us repeat that for a cooling of -45 deg.C !!
delta_T=-45.0 # uniform cooling
param.update({'radp':0.0,'aoht':0.0,'falb':2,'restart':0})
d_olr,d_asr=response_initial_perturbation(param,delta_T)

# Feedback analysis

# can you find other stable equilibria ?

In [ ]:
# (h) Sensitivity to solar constant

# first cool
S0array = np.arange(1400., 1099, -20)
param.update({'radp':0.0,'ifig':0,'restart':0,'falb':2})
resu_cool=ebm_solar_sensitivity(param,S0array)

# then warm from end state of the cooling experiment
S1array = np.flipud(S0array)
sati=resu_cool['sat_eq'][:,-1]
ssti=resu_cool['sst_eq'][:,-1]
param.update({'radp':0.0,'ifig':0,'restart':1,'falb':2,'ssti':ssti,'sati':sati})
resu_warm=ebm_solar_sensitivity(param,S1array)

# make figure
make_fig_cool_warm(S0array,S1array,resu_cool,resu_warm)

In [ ]:
#   Now unfreeze the snowball state !!
#   For which S0 value can we escape the snowball ?
#   Run the EBM for one S0 value
#   Take the initial conditons (SAT and SST) corresponding to the equilibrium state obtained
#   with S0=1100 W/m2 (--> initial conditions = snowball state) 

In [ ]:
sati=resu_cool['sat_eq'][:,-1] # initial condition SAT = end state of experiment with S0=1100 W/m2
ssti=resu_cool['sst_eq'][:,-1] # initial condition SST = end state of experiment with S0=1100 W/m2
# S0= # transition to ice-free state occurs at S0= ????? W/m2
param.update({'S0':S0,'radp':0.0,'restart':1,'sati':sati,'ssti':ssti,'falb':2})
resu_hot=ebm_annual(param)

In [ ]:
# warm from S0=1100 to S0=value larger than the critical S0 value found previously

# S2array = np.linspace(1100., ???? , 19)
sati=resu_cool['sat_eq'][:,-1]
ssti=resu_cool['sst_eq'][:,-1]
param.update({'dt':2.0,'radp':0.0,'ifig':0,'restart':1,'sati':sati,'ssti':ssti,'falb':2})
resu_warm_bis=ebm_solar_sensitivity(param,S2array)

# make figure
make_fig_cool_warm(S0array,S2array,resu_cool,resu_warm_bis)


In [ ]:
# transition at about S0= ???? (value found previously)
# transition from snowball to ice-free state occurs when max(SST) > Tf
# with Tf=-10 deg.C the freezing temperature (below Tf ice is present with high albedo 0.62)
sati=resu_cool['sat_eq'][:,-1]
ssti=resu_cool['sst_eq'][:,-1]
# below we run the model for 3 different values of S0 around the critical value found previously
# S0= ???? # for example take a value 2 W/m2 below the critical value
param.update({'S0':S0,'radp':0.0,'restart':1,'sati':sati,'ssti':ssti,'falb':2})
resu_hot_1=ebm_annual(param)
# S0= ??? # for example take a value 1 W/m2 below the critical value
param.update({'S0':S0,'radp':0.0,'restart':1,'sati':sati,'ssti':ssti,'falb':2})
resu_hot_2=ebm_annual(param)
# S0= ??? # critical value
param.update({'S0':S0,'radp':0.0,'restart':1,'sati':sati,'ssti':ssti,'falb':2})
resu_hot_3=ebm_annual(param)

# figure showing the time evolution of average SAT and SST for those 3 different values of S0
plt.figure(figsize=(12,10))
plt.subplot(2,1,1)
plt.plot(resu_hot_1['time'][:-1],resu_hot_1['satmean'],'b',label='S_0=XXXX W/m2')
plt.plot(resu_hot_2['time'][:-1],resu_hot_2['satmean'],'r',label='S_0=XXXX W/m2')
plt.plot(resu_hot_2['time'][:-1],resu_hot_3['satmean'],'k',label='S_0=XXXX W/m2')
plt.legend(loc='best',fontsize=14)
plt.ylabel('deg.C',fontsize=14)
plt.title('Global average SAT',fontsize=14)
plt.subplot(2,1,2)
Tf=-10.0
plt.plot(resu_hot_1['time'],np.amax(resu_hot_1['sst'],axis=0),'b',label='S_0=XXXX W/m2')
plt.plot(resu_hot_2['time'],np.amax(resu_hot_2['sst'],axis=0),'r',label='S_0=XXXX W/m2')
plt.plot(resu_hot_2['time'],np.amax(resu_hot_3['sst'],axis=0),'k',label='S_0=XXXX W/m2')
plt.plot([0,nyears],[Tf,Tf],'k--',label='Freezing temperature')
plt.legend(loc='best',fontsize=14)
plt.xlabel('Time (years)',fontsize=14)
plt.ylabel('deg.C',fontsize=14)
plt.title('Maximum SST',fontsize=14)
plt.show(block=False)